In [6]:
import numpy as np
import pandas as pd

from library import INITIAL_PERMUTATION_TABLE, INVERSE_PERMUTATION_TABLE, PC1_TABLE, PC2_TABLE, EXPANSION_TABLE, SBOX, PERMUTATION_TABLE


import textwrap

### Açık metine karşılık gelen ikili kod

In [63]:
def string_to_binary(plaintext):
    binary_plaintext = ''.join(format(ord(i),'b').zfill(8) for i in plaintext) 
    return binary_plaintext

In [99]:
binary_plaintext = string_to_binary("kemaloze")
bp = []

for i in range(64):
    bp.append(binary_plaintext[i])

In [98]:
BP = pd.DataFrame([bp[0:8],
                bp[8:16],
                bp[16:24],
                bp[24:32],
                bp[32:40],
                bp[40:48],
                bp[48:56],
                bp[56:64]])
BP

,0,1,2,3,4,5,6,7
0,0,1,1,0,1,0,1,1
1,0,1,1,0,0,1,0,1
2,0,1,1,0,1,1,0,1
3,0,1,1,0,0,0,0,1
4,0,1,1,0,1,1,0,0
5,0,1,1,0,1,1,1,1
6,0,1,1,1,1,0,1,0
7,0,1,1,0,0,1,0,1


### IP uygulanması

![IP](http://www.umsl.edu/~siegelj/information_theory/projects/des.netau.net/images/ip2.png)

In [70]:
def apply_initial_permutation(binary_plaintext,INITIAL_PERMUTATION_TABLE):
    permutated_text = ""
    for index in INITIAL_PERMUTATION_TABLE:
        permutated_text += binary_plaintext[index-1]
    return permutated_text

### L0 ve R0 ayrımı

In [22]:
def split_in_half(binarybits):
    L0 = binarybits[:32]
    R0 = binarybits[32:]
    return L0,R0

## F FONKSİYONU

#### R0'a Expansion Table uygulanması

In [25]:
def apply_expansion(R0,EXPANSION_TABLE):
    bits48 = ""
    for index in EXPANSION_TABLE:
        bits48 += R0[index-1]
    return bits48

#### XOR (K0 ile)

In [44]:
def XOR(bits1,bits2):
	xor_result = ""
	for index in range(len(bits1)):
		if bits1[index] == bits2[index]: 
			xor_result += '0'
		else:
			xor_result += '1'
	return xor_result

#### SBOX

In [52]:
'''6 bitlik parçalara bölme'''

def split_in_6bits(XOR_48bits):
	list_of_6bits = textwrap.wrap(XOR_48bits,6)
	return list_of_6bits

In [53]:
'''İlk ve son bitin alınması'''

def get_first_and_last_bit(bits6):
	twobits = bits6[0] + bits6[-1] 
	return twobits

In [54]:
'''Ortadaki 4 bitin alınması'''

def get_middle_four_bit(bits6):
	fourbits = bits6[1:5] 
	return fourbits

In [55]:
'''İkili kodun 10luk sistemdeki karşılığı'''

def binary_to_decimal(binarybits):
	decimal = int(binarybits,2)
	return decimal

In [49]:
'''10luk sistemdeki sayının ikili karşılığı'''

def decimal_to_binary(decimal):
	binary4bits = bin(decimal)[2:].zfill(4)
	return binary4bits

In [56]:
'''Satır ve sütun numaralarından sbox karşılığı'''

def sbox_lookup(sboxcount,first_last,middle4):
	d_first_last = binary_to_decimal(first_last)
	d_middle = binary_to_decimal(middle4)
	
	sbox_value = SBOX[sboxcount][d_first_last][d_middle]
	return decimal_to_binary(sbox_value)

#### Permutation table

In [ ]:
def apply_permutation(sbox_32bits,PERMUTATION_TABLE):
	
	final_32bits = ""
	for index in PERMUTATION_TABLE:
		final_32bits += sbox_32bits[index-1]
	return final_32bits

### F fonksyonunun uygulanması

In [58]:
def functionF(pre32bits, key48bits):	
	result = ""
	expanded_left_half = apply_expansion(pre32bits)
	xor_value = XOR(expanded_left_half,key48bits)
	bits6list = split_in_6bits(xor_value)
	for sboxcount, bits6 in enumerate(bits6list):
		first_last = get_first_and_last_bit(bits6)
		middle4 = get_middle_four_bit(bits6)
		sboxvalue = sbox_lookup(sboxcount,first_last,middle4)
		result += sboxvalue
	final32bits = apply_permutation(result)	
	return final32bits

### ANAHTAR OLUŞTURMA

#### PC1

![IP](http://www.umsl.edu/~siegelj/information_theory/projects/des.netau.net/images/ip2.png)

In [59]:
def apply_pc1(key_64bits,PC1_TABLE):
    key_56bits = ""
    for index in PC1_TABLE:
        key_56bits += key_64bits[index-1]
    return key_56bits

#### Left Key ve Right Key ayrımı

In [60]:
def split_in_half(key_56bits):
    left_key = key_56bits[:28]
    right_key = key_56bits[28:]
    return left_key,right_key

#### Circular Left Shift

In [ ]:
def circular_left_shift(left_key,right_key):
    shifted_left = left_key[1:] + left_key[:1]
    shifted_right = right_key[1:] + right_key[:1]
    shifted = shifted_left + shifted_right
    return shifted

#### PC2

In [61]:
def apply_pc2(shifted,PC2_TABLE):
    key_48bits = ""
    for index in PC2_TABLE:
        keys_48bits += shifted[index-1]
    return key_48bits

### Anahtarın oluşturulması

In [71]:
def generate_key(key_64bits):
    pc1_out = apply_pc1(key_64bits) 
    C0,D0 = split_in_half(pc1_out)
    shifted = circular_left_shift(C0,D0)
    key = apply_pc2(shifted)
    return key